In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import os

import random

from collections import defaultdict

import torch
import torch.nn as nn
import transformers
import re

from transformers import (AutoTokenizer,
                          AutoModelForCausalLM)

import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

from peft import (PeftModel,
                  PeftConfig)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# put your file path here
path_to_nlu_dir = "/content/drive/MyDrive/NLU/Final Project/NLU_FinalProject/"

data_dir = path_to_nlu_dir+"Data/JSONL_Formatted/"
data_path = "RACE-H/RACE-H_v1_tst.jsonl"
data_name = 'RACE-H_FineTuneV1_InfoAlterationSyn_test' #InfoAlterationSyn_test'
#data_path = "SAT_ACT/SATACT_v3_tst.jsonl"
#data_name = 'SATACT_FineTuneV1_InfoAlterationSent_test'

save_dir = path_to_nlu_dir+"Results/v2_Results/"

#model_name = "gpt2-xl" #baseline
#model_name = "Salm00n/gpt2-xl_RACE-H_v1" #v1 race-h fine-tuned
#model_name = "Salm00n/gpt2-xl_SATACT_v1" #v1 sat/act fine-tuned
model_name = "Salm00n/gpt2-xl_RACE-H_v2" #v2 race-h fine-tuned
#model_name = "Salm00n/gpt2-xl_SATACT_v2" #v2 sat/act fine-tuned
#model_name = "Salm00n/gpt2-xl_RACE-H_v3" #v3 race-h fine-tuned
#model_name = "Salm00n/gpt2-xl_SATACT_v3" #v3 sat/act fine-tuned
BATCH_SIZE = 1
random.seed(0)

In [ ]:
#mode = 0 #baseline
mode = 1 #baseline with input alteration (synonyms)
#mode = 2 #baseline with input alteration (added sentence)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if 'Salm00n' in model_name: # LoRA fine-tuned model
  config = PeftConfig.from_pretrained(model_name)
  base_model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                                    torch_dtype=torch.float16)
  model = PeftModel.from_pretrained(base_model, model_name, torch_dtype=torch.float16)
  model = model.merge_and_unload()
else: # base model
  model = AutoModelForCausalLM.from_pretrained(model_name)

model.to(device)
model.eval()

#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("gpt2-xl")
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id,
                              reduction="none")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Load and Process Data

In [ ]:
with open(data_dir + data_path, 'r') as f:
  data = [json.loads(line) for line in f]

print(f'Number of Questions = {len(data)}\n')
print(data[0].keys())

Number of Questions = 3498

dict_keys(['context', 'question', 'answerA', 'answerB', 'answerC', 'answerD', 'correct'])


In [ ]:
#Add code to randomly replace words with their synonyms - see NLP w RL HW3
def custom_transform(dict_text):
    #text should be a dictionary value - ex: data["context"]
    replace_prob = 0.3 # change to desired value
    words = word_tokenize(dict_text)
    new_words = []

    for word in words:
        # if random number is less than threshold, replace word with synonym
        if random.random() < replace_prob:
            # get synsets for word
            synsets = wordnet.synsets(word)

            # get lemmas (synonym words) from synsets
            synonyms = set(lemma.name() for syn in synsets for lemma in syn.lemmas() if lemma.name().lower() != word.lower())

            # replace the word with synonym
            if synonyms:
                new_word = random.choice(list(synonyms)) #replace with highest/higher prob synonym
                new_words.append(new_word)
            else:
                new_words.append(word)  # no change if no synonyms
        # if random number is >= threshold, keep original word
        else:
            new_words.append(word)

    # recreate sentence with/without synonyms
    augmented_text = TreebankWordDetokenizer().detokenize(new_words)

    return augmented_text

#Just passage OR passage, question, answer choices?

In [ ]:
# get random sentence of text
def get_random_sentence(text):
    sentences = re.split(r'(?<=[.!?]) +', text)
    return random.choice(sentences)

In [ ]:
def create_input(batch, sys_prompt='', fs_demos='', mode=mode):
  texts = []
  if mode == 0:
    print("Processing standard data")
    for text in batch:
      pqa = [f"{fs_demos}Q: {text['context']} {text['question']}\nA:{sys_prompt} {text[i]}"
              for i in ['answerA', 'answerB', 'answerC', 'answerD']]
      texts.extend(pqa)

  elif mode == 1:
    print("Processing augmented data - synonyms")
    for text in batch:
      pqa = [f"{fs_demos}Q: {custom_transform(text['context'])} {custom_transform(text['question'])}\nA:{sys_prompt} {custom_transform(text[i])}"
              for i in ['answerA', 'answerB', 'answerC', 'answerD']]
      texts.extend(pqa)

  elif mode == 2:
    print("Processing augemented data - sentence")
    rand_sent = "The sky is blue on Saturdays."
    for text in batch:
      pqa = [f"{fs_demos}Q: {text['context']} {rand_sent} {text['question']}\nA:{sys_prompt} {text[i]}"
              for i in ['answerA', 'answerB', 'answerC', 'answerD']]
      texts.extend(pqa)
      rand_sent = get_random_sentence(text['context'])

  return texts

create_input(data[:BATCH_SIZE])

Processing augmented data - synonyms


['Q: It is the finish of politicians everywhere---- -how to succeed and keep the trust of voters . Now researchers at the University of St Anurew\'s in Scotland say they may have the answer . They believe politician could learn a lot from recent march_on in science . A growing number of studies have shown that people do judge a book by its cover . researcher enounce about of us make quick judgments about a person on the base of how they look . analyse suggest that people are to_a_lesser_extent likely to trust those with in_particular masculine feature, such vitamin_A adenine square jaw, pocket-sized eyes or a big nose. "They ar considered dominant and to_a_lesser_extent trustworthy," order Ms Cornwell . "information_technology doesn\'t signify that men World_Health_Organization look to_a_greater_extent masculine are to_a_lesser_extent trustworthy---- -it\'s just our first impression ." Those with less masculine sport---- -larger eyes, a smaller nose and thinner lip are consider to be m

Run Inference

In [ ]:
def batch_predict(batch_processed):
  input = tokenizer(batch_processed, padding=True, return_tensors='pt').to(device)

  # truncating if needed (truncate front of input)
  if input['input_ids'].shape[-1] > 1024:
    trunc = [True]* (len(batch_processed)//4)
    input['input_ids'] = input['input_ids'][:,-1024:]
    input['attention_mask'] = input['attention_mask'][:,-1024:]
  else:
    trunc = [False]* (len(batch_processed)//4)

  with torch.no_grad():
    output = model(**input, max_new_tokens=0)

  # get prediction with CrossEntropyLoss
  logits = output.logits[:, :-1, :].to('cpu')
  targets = input['input_ids'][:, 1:].to('cpu')

  loss = loss_fn(logits.permute(0, 2, 1), targets)
  total_loss = torch.sum(loss, dim=-1)
  total_loss = total_loss.view(-1, 4)

  preds = torch.argmin(total_loss, dim=-1).tolist()

  return trunc, total_loss.numpy(), preds

In [ ]:
# test output
example_batch = data[:BATCH_SIZE]
batch_predict(create_input(example_batch))

Processing augmented data - synonyms


([False], array([[1973., 1958., 2022., 2062.]], dtype=float16), [1])

In [ ]:
res = defaultdict(list)

for i in tqdm(range(0, len(data), BATCH_SIZE)):
  batch = data[i:i+BATCH_SIZE]
  try:
    trunc, total_loss, preds = batch_predict(create_input(batch))
  except Exception as e:
    print(e)
    trunc = [np.nan]*BATCH_SIZE
    preds = [np.nan]*BATCH_SIZE
    total_loss = np.full((BATCH_SIZE, 4), np.nan)

  res['pred'].extend(preds)
  res['truncated'].extend(trunc)
  res['loss_A'].extend(total_loss[:,0].tolist())
  res['loss_B'].extend(total_loss[:,1].tolist())
  res['loss_C'].extend(total_loss[:,2].tolist())
  res['loss_D'].extend(total_loss[:,3].tolist())

  0%|          | 0/3498 [00:00<?, ?it/s]

Processing augmented data - synonyms


  0%|          | 1/3498 [00:01<1:14:01,  1.27s/it]

Processing augmented data - synonyms


  0%|          | 2/3498 [00:02<1:25:50,  1.47s/it]

...

100%|█████████▉| 3496/3498 [1:34:56<00:03,  1.54s/it]

Processing augmented data - synonyms


100%|█████████▉| 3497/3498 [1:34:57<00:01,  1.54s/it]

Processing augmented data - synonyms


100%|██████████| 3498/3498 [1:34:59<00:00,  1.63s/it]


In [ ]:
res_df = pd.concat([pd.DataFrame(res), pd.DataFrame(data)], axis=1)
res_df['pred'] = res_df['pred'].apply(lambda x: 'ABCD'[x])
res_df.head()

,pred,truncated,loss_A,loss_B,loss_C,loss_D,context,question,answerA,answerB,answerC,answerD,correct
0,C,False,2062.0,2092.0,1818.0,2026.0,It is the goal of politicians everywhere-----h...,"According to the passage, we know that _ .",people with good facial features must be trust...,people with bad facial features could not be t...,we should judge people by their facial features,facial features might give people some wrong i...,D
1,B,False,1946.0,1804.0,2086.0,1990.0,It is the goal of politicians everywhere-----h...,"According to Ms Cornwell, we can infer that ...",the science will give politicians great help,politicians could be successful with the help ...,politicians won't think highly of the science,politicians will be satisfied with the science,C
2,B,False,2015.0,1786.0,1843.0,1970.0,It is the goal of politicians everywhere-----h...,What's the best title for the passage?,How Science could Help Politicians,How to Win the Trust of Voters,The Other Sides of Politicians,An Important Discovery for Politicians,A
3,D,False,2740.0,2556.0,2556.0,2496.0,"In the 1960s, people asked about your astrolog...",The main purpose of the passage is to tell you...,what a website is like,how to build your own website,how to meet people online,what a website is made up of,B
4,A,False,2252.0,2684.0,2502.0,2534.0,"In the 1960s, people asked about your astrolog...","According to the writer, your website is a pla...",where you can meet people all around the world,where you can buy what you want,where you can get free services,where you can meet people on the Internet,D


In [ ]:
print(res_df.pred.value_counts())

pred
A    921
D    861
B    860
C    856
Name: count, dtype: int64


In [ ]:
# evaluation metrics

# accuracy
accuracy = sum(res_df.pred == res_df.correct)/len(res_df.dropna())

# percent failure
res_fail = sum(res_df.pred.isnull())/len(res_df)

# percent truncated
res_trunc = sum(res_df.truncated)/len(res_df)

df_eval = pd.DataFrame({'dataset':[data_name],
                        'accuracy':[accuracy],
                        'response_failure':[res_fail],
                        'prompt_truncation':[res_trunc]})

if os.path.exists(f"{save_dir}benchmark_summary_{model_name}_mode{mode}_v2.csv"):
  df_temp = pd.read_csv(f"{save_dir}benchmark_summary_{model_name}_v2.csv")
  df_eval = pd.concat([df_eval, df_temp], axis=0, ignore_index=True)

df_eval

,dataset,accuracy,response_failure,prompt_truncation
0,RACE-H_FineTuneV1_InfoAlterationSyn_test,0.248999,0.0,0.009434


Store Results

In [ ]:
if '/' in model_name:
  model_name = model_name.split('/')[1]
  print(model_name)
res_df.to_csv(f"{save_dir}{data_name}_benchmark_{model_name}_v2_mode{mode}.csv", index=False)

gpt2-xl_RACE-H_v2


In [ ]:
df_eval.to_csv(f"{save_dir}{data_name}benchmark_summary_{model_name}_v2_mode{mode}.csv", index=False)